In [ ]:
import string
from string import punctuation
from os import listdir
from collections import Counter
import re
import unicodedata

import pandas as pd
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

On reprend les données pré-traitées précédemment.

In [ ]:
df = pd.read_pickle("clean_data.pkl")

In [3]:
df.head(10)

,Reviews,Tidy_Reviews,label
32,This is the worst thing the TMNT franchise has...,bad thing tmnt franchise ever spawn kid come s...,1
8456,Shame on Julia Roberts and John Cusack. They a...,shame julia robert john cusack talented part s...,1
1650,A poorly written script with no likeable chara...,poorly write script likeable comedy forgot lau...,1
8101,Bo Derek's beauty and John Derek's revolutiona...,bo derek beauty john derek revolutionary direc...,0
4908,For that matter one of the worst FILMS ever ma...,matter bad ever plot follow slog jungle look a...,1
57,Caught the tail end of this movie channel surf...,caught tail end channel surf cable channel int...,0
1363,"the more i think about it, there was nothing r...",think nothing redeem saw month ago memory migh...,1
1503,"I am a big fan of the Spaghetti Western Genre,...",big fan spaghetti western genre usually also l...,1
9557,Remembering the dirty particulars of this insi...,remember dirty particular insidiously vapid ak...,1
5968,Typical De Palma movie made with lot's of styl...,typical de palma lot style bring edge certainl...,0


In [4]:
print(df["Reviews"][32])

32    This is the worst thing the TMNT franchise has...
32    This interesting Giallo boosts a typical but s...
Name: Reviews, dtype: object


On sépare les données en données train et en données test.

In [5]:
from sklearn.model_selection import train_test_split
reviews_train, reviews_test, label_train, label_test = train_test_split(df["Tidy_Reviews"], df["label"], test_size=0.4, random_state=42)

On sépare les données train en données de validation et en données train.

In [6]:
data_train, data_val, y_train, y_val = train_test_split(reviews_train, label_train, test_size=0.2, random_state=42)

In [7]:
data_train = data_train.to_numpy()
data_val = data_val.to_numpy()
reviews_train = reviews_train.to_numpy()

On fait un vocabulaire seulement sur le train.

In [36]:
import nltk    
voc = Counter()
for review in data_train:
    tokens = nltk.word_tokenize(review)
    voc.update(tokens)

In [37]:
voc.update(["<pad>","<unk>"])

In [38]:
vocab = sorted(voc, key=voc.get, reverse=True)

In [39]:
len(vocab)

43452

In [40]:
vocab_to_int = {word: ii for ii, word in enumerate(voc, 1)}

In [41]:
## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = [] 
for review in data_train:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [42]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))
# should ~ 74000+ print() 
# print tokens in first review 
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  43452
Tokenized review: 
 [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 10, 14, 15, 16, 17, 1, 18, 19, 6, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 7, 8, 30, 31, 32, 33, 34, 35, 10, 36, 37, 4, 38, 38, 39, 40, 29, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 13, 53, 54, 55, 56, 57, 58, 59, 60, 61]]


In [43]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 1360


In [44]:
print('Number of reviews before removing outliers: ', len(reviews_ints))  ## remove any reviews/labels with zero length from the reviews_ints list.
# get indices of any reviews with length 0 non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]
# remove 0-length reviews and their labels reviews_ints = [reviews_ints[ii] for ii in non_zero_idx] labels = np.array([labels[ii] for ii in non_zero_idx])  
print('Number of reviews after removing outliers: ', len(reviews_ints))      

Number of reviews before removing outliers:  10897
Number of reviews after removing outliers:  10897


In [46]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0'sor truncated to the input seq_length.'''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    # for each review, I grab that review and
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    return features# Test your implementation!

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:30,:10])

[[   0    0    0    0    0    0    0    0    0    0]
 [  62   63   64   65   66   67   68   69   70   71]
 [ 256  257  258  259  260  261  262  179  263  264]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0

In [47]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = y_train[:split_idx], y_train[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(8717, 200) 
Validation set: 	(1090, 200) 
Test set: 		(1090, 200)


In [48]:
train_x = np.array(train_x)
train_y = np.array(train_y)
val_x = np.array(val_x)
val_y = np.array(val_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

In [49]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [50]:
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, batch_size=50)

In [51]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [52]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,  1397,    14,    29],
        [    0,     0,     0,  ...,  1886,   874,  2020],
        [  714,   962,   672,  ...,  1952, 28492,  1701],
        ...,
        [    0,     0,     0,  ...,   236,    21,    41],
        [    0,     0,     0,  ...,   503,     2,  1159],
        [    0,     0,     0,  ...,  2512, 19462,  5795]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 1])


In [53]:
# obtain one batch of training data
dataiter = iter(valid_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,    15,  3310,  3310],
        [    0,     0,     0,  ...,   253,   613,   445],
        [    0,     0,     0,  ...,    13,    27,  9525],
        ...,
        [    0,     0,     0,  ...,  7102,   236,   682],
        [    0,     0,     0,  ..., 12462, 10308, 18897],
        [   15,  1602,   735,  ..., 13350,   149,  5659]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1])


In [54]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [55]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))
  

In [56]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [57]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,437,105 trainable parameters


In [58]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [59]:
criterion = nn.BCEWithLogitsLoss()

In [60]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [79]:
def train(model, iterator, optimizer, criterion):
    
    if(train_on_gpu):
        model.cuda()

    model.train()
    
    epoch_loss = 0
    epoch_acc = 0   
    counter = 0
      
    for inputs, labels in train_loader:
        counter += 1
        
        optimizer.zero_grad()
        
        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
         
        inputs = torch.transpose(inputs, 0, 1)
        
        predictions = model.forward(inputs).squeeze(1) 
        
        labels = labels.float()
        
        loss = criterion(predictions, labels)
        
        acc = binary_accuracy(predictions, labels)
    
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
          
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [80]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    counter = 0    
    model.eval()
    
    with torch.no_grad():
    
        for inputs, labels in valid_loader:
            counter += 1

            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            inputs = torch.transpose(inputs, 0, 1)

            predictions = model.forward(inputs).squeeze(1) 

            labels = labels.float()

            loss = criterion(predictions, labels)

            acc = binary_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [81]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [82]:
N_EPOCHS = 3
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    # initialize hidden state
    #h = model.init_hidden(batch_size)
    
    train_loss, train_acc = train(model, dataiter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, dataiter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

0.5399999618530273
0.9799999594688416
1.4199999570846558
1.9999999403953552
2.5799999237060547
3.119999885559082
3.679999887943268
4.259999871253967
4.839999854564667
5.339999854564667
5.979999840259552
6.5599998235702515
6.97999981045723
7.619999796152115
8.159999758005142
8.599999755620956
9.11999973654747
9.639999717473984
10.379999727010727
10.919999688863754
11.419999688863754
11.944999724626541
Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 5.419 | Train Acc: 447.50%
	 Val. Loss: 0.684 |  Val. Acc: 54.30%
0.47999998927116394
0.9399999678134918
1.539999932050705
2.0799998939037323
2.7199998795986176
3.2799998819828033
3.7199998795986176
4.1999998688697815
4.6999998688697815
5.239999830722809
5.699999809265137
6.359999775886536
6.899999737739563
7.43999969959259
7.93999969959259
8.559999704360962
9.079999685287476
9.819999694824219
10.339999675750732
10.79999965429306
11.359999656677246
11.959999680519104
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 5.414 | Train Acc: 449.33%
	 Val. Loss

In [65]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, dataiter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.691 | Test Acc: 52.84%


In [151]:
# Get test data loss and accuracy

net = model 
test_losses = [] # track loss
num_correct = 0

# init hidden state
#h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    #h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    inputs = torch.transpose(inputs, 0, 1)
    # get predicted outputs
    output = net(inputs)
    

    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
 
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select

# Test

In [40]:
test_reviews = df["Reviews"][1:2]
test_reviews

8456    Shame on Julia Roberts and John Cusack. They a...
Name: Reviews, dtype: object

In [114]:
from string import punctuation

def tokenize_review(test_review, voc):
    
    test_ints = []
    
    for review in test_review:
        # lowercase
        review = review.lower() 
        
        # get rid of punctuation
        test_text = ''.join([c for c in review if c not in punctuation])

        # splitting by spaces
        test_words = test_text.split()

        # tokens       
        test_ints.append([vocab_to_int[word] if word in voc else vocab_to_int["<unk>"] for word in test_words ])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_pos, voc)
print(test_ints)

[[699, 699, 699, 699, 41, 21340, 699, 699, 194, 699, 23622, 236, 699, 699, 699]]


In [115]:
# test sequence padding
seq_length=200
features = pad_features(test_ints, seq_length)

print(features)

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [116]:
# test conversion to tensor and pass into your model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

torch.Size([1, 200])


In [149]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review,voc)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    #h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    feature_tensor = torch.transpose(inputs, 0, 1)
    
    output = net(feature_tensor)
    
    # get the output from the model
    #output = net(feature_tensor)
    print(output)
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    print(pred)
    # printing output value, before rounding
    #print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

    # positive test review
test_review_pos = ['This movie had the best acting and the dialogue was so good. I loved it.']

In [150]:
# call function
seq_length=200 # good to use the length that was trained on

predict(model, test_review_pos, seq_length)

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select

In [39]:
freq_words = pd.DataFrame({'Word':list(voc.keys()),'Count':list(voc.values())})
freq_words.head(10)

,Word,Count
0,like,10155
1,say,4492
2,twice,220
3,love,3920
4,act,3777
5,great,4770
6,even,6043
7,though,2189
8,similar,421
9,plot,3314


In [98]:
freq_words.shape

(43453, 2)

In [42]:
import torch
from torchtext import data

In [214]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(data_train), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(data_val), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(reviews_test), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.

In [215]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 1

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (data_train, data_val, reviews_test),
    sort = False, #don't sort test/validation data
    batch_size=BATCH_SIZE,
    device=device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (data_train, data_val, reviews_test),
    sort_key = lambda x: x.s, #sort by s attribute (quote)
    batch_size=BATCH_SIZE,
    device=device)

print('Train:')
for batch in train_iterator:
    print(batch)
    
print('Valid:')
for batch in valid_iterator:
    print(batch)
    
print('Test:')
for batch in test_iterator:
    print(batch)

AttributeError: 'numpy.ndarray' object has no attribute 'sort_key'